In [167]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow_hub as hub

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import GlobalMaxPooling1D

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [218]:
train_limpio = pd.read_csv("train_limpio_con_BOW_de_5000_y_Stemming_2.csv",encoding = "ISO-8859-1", dtype={"text": "array"})
test_limpio = pd.read_csv("test_limpio_con_BOW_de_5000_y_Stemming_2.csv",encoding = "ISO-8859-1")

TypeError: data type 'array' not understood

In [212]:
train_limpio["text"].head(1)

0    [   0    0    0    0    0    0    0    0    0 ...
Name: text, dtype: object

In [170]:
VALIDATION_SIZE = .8
TEST_SIZE = .9
df = train_limpio
raw_train_df, raw_val_df, raw_test_df = np.split(df.sample(frac=1), [int(VALIDATION_SIZE*len(df)), int(TEST_SIZE*len(df))])

In [171]:
raw_train_df_no_target = raw_train_df.drop(columns=["target", "objetividad", "sentimiento"])
raw_val_df_no_target = raw_val_df.drop(columns=["target", "objetividad", "sentimiento"])
raw_test_df_no_target = raw_test_df.drop(columns=["target", "objetividad", "sentimiento"])

raw_train_df_target = raw_train_df[["target"]]
raw_val_df_target = raw_val_df[["target"]]
raw_test_df_target = raw_test_df[["target"]]

In [196]:
raw_train_df_no_target["text"].head(1).replace("\n", "").to_numpy()

array(['[   0    0    0    0    0    0    0    0    0    0    0    0    0    0\n    0    0    0    0    0    0    0    0    0    0    0    0    0    0\n    0    0    0    0    0    0    0    0    0    0    0    0    8  131\n  641    5 2518  460  317  536    1   51]'],
      dtype=object)

In [173]:
#len(list(raw_train_df_no_target.to_numpy(dtype="float32"))[0])

In [176]:
vocab_size = 10000
embedding_dim = 16
max_length = 50
trunc_type='post'
oov_tok = "<OOV>"

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
num_epochs = 10
model.summary()
history_conv = model.fit(raw_train_df_no_target["text"], raw_train_df_target, epochs=num_epochs, validation_data=(raw_val_df_no_target["text"], raw_val_df_target))

Model: "sequential_77"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 50, 64)            640000    
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 46, 128)           41088     
_________________________________________________________________
global_average_pooling1d_43  (None, 128)               0         
_________________________________________________________________
dense_144 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_145 (Dense)            (None, 1)                 65        
Total params: 689,409
Trainable params: 689,409
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


ValueError: in user code:

    /home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.

In [ ]:
model1.evaluate(raw_test_df_no_target.to_numpy(dtype="uint"), raw_test_df_target.to_numpy(dtype="uint"))